# Uvod
Ovaj se projekt bavi specifičnim problemom pri računanju PageRanka, a to je problem takozvanih visećih čvorova.
Kao što bi čitatelj već trebao biti upoznat, PageRank algoritam uzima u obzir poveznice koje pokazuju s jedne web stranice na drugu.
Koristeći te poveznice i element personalizacije, konstruira se stohastička matrica kojoj se zatim računa stacionarna distribucija.
Pri stvaranju web stranica nitko ne sprečava autora te stranice da umetne linkove drugih stranica na svoju. Time nastaju viseći čvorovi zbog kojih se gubi svojstvo stohastičnosti matrica, tj. da je suma svakog retka jednaka $1$. 
Neka matrica $H$ predstavlja matricu povezanosti web stranica.
$$H = 
\begin{pmatrix}
H_{1,1} & H_{1,2} \\
0 & 0
\end{pmatrix}
$$

$$H = 
\begin{pmatrix}
0 & 0 & 1 & 0 & 0\\
\frac{1}{2} & 0 & 0 & 0 & \frac{1}{2}\\
\frac{1}{2} & \frac{1}{4} & \frac{1}{4} & 0 & 0\\
0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0
\end{pmatrix}
$$

<center>
    <img src="images/graph.png" width="600">
</center>

Kada bismo pokušali stvoriti Googleovu matricu $G$
$$G = \alpha H + (1 - \alpha) E$$
$G$, ona ne bi bila stohastička matrica pa ćemo stoga matricu $H$ zamijeniti matricom $S$.
$$S =
\begin{pmatrix}
H_{1,1} & H_{1,2} \\
ew_1^T & e w_2^T
\end{pmatrix} 
\quad
w = \begin{bmatrix} w_1 \\ w_2 \end{bmatrix}
$$

Gdje je $e$ vektor popunjen jedinicama, a $w$ je vektor čiji se elementi sumiraju u $1$.
Drugim riječima, uzeti ćemo vektor $w$ i popuniti nul retke matrice $H$ njime.
Matrica $E$ se računa na temelju onoga što je korisnik upisao u tražilicu
$$E = e v^T$$
gdje je $v$ tkz. personalizacijski vektor.

# Optimizacija postupka računa stacionarnih distribucija
Možemo značajno ubrzati računanje stacionarne distribucije tako da iskoristimo činjenicu
da se $n- k$ redaka ponavlja, gdje je $n$ ukupan broj stranica (poveznica), a $k$ je broj visećih stranica. To ćemo napraviti uz pomoć matrice $X$ (njezina definicija je opisana u literaturi). Za koju vrijedi:
$$XGX^{-1} = 
\begin{pmatrix}
G^{(1)} & * \\
0 & 0
\end{pmatrix}
$$
$G^{(1)}$ se lako računa pomoću vektora $u = \begin{bmatrix} u_1 \\ u_2 \end{bmatrix} =  \alpha w + (1 - \alpha)v$ 
$$ G^{(1)} =
\begin{bmatrix}
G_{11} & G_{12}e \\
u_1^T & u_2^T e
\end{bmatrix}
$$
Pomoću teorema koji su dani u literaturi dovoljno je izračunati stacionarnu distribuciju matrice $G^{(1)}$ koja je
značajno manja od $G$ (ako ima puno visećih čvorova).

Računamo:
$$
\sigma^T
\begin{bmatrix}
G_{11} & G_{12}e \\
u_1^T & u_2^T e
\end{bmatrix}
= \sigma^T,
\quad \sigma \geq 0,
\quad \|\sigma\| = 1
$$

Neka je $ \sigma^T = \begin{bmatrix} \sigma_{1:k}^T & \sigma_{k+1} \end{bmatrix} $, gdje je $ \sigma_{k+1} $ skalar.

Tada PageRank vektor možemo dobiti iz:

$$
\pi^T =
\begin{bmatrix}
\sigma_{1:k}^T &
\sigma^T
\begin{pmatrix}
G_{12} \\
u_2^T
\end{pmatrix}
\end{bmatrix}
$$
Ovako smo izbjegli računanje stacionarne distribucije velike matrice $G$.

# Algoritam 

% Ulazi: $H, v, w, \alpha$ $\quad$ Izlaz: $\hat{\pi}$

% Metoda potencija primijenjena na $G^{(1)}$:

Odaberi početni vektor $\hat{\sigma}^{T} = [\hat{\sigma}^{T}_{1:k} \quad \hat{\sigma}_{k + 1}]$ sa $\hat{\sigma} \ge 0$, $||\hat{\sigma}|| = 1$.

$\textbf{Dok}$ nije konvergiralo:

$\qquad \hat{\sigma}^{T}_{1:k} = \alpha \hat{\sigma}^{T}_{1:k} H_{11} + (1-\alpha)v_1^T + \alpha \hat{\sigma}_{k + 1} w_1^T$

$\qquad \hat{\sigma}_{k + 1} = 1 - \hat{\sigma}^{T}_{1:k} e$

$\textbf{zavrsi dok}$

% Vrati PageRank

$\hat{\pi}^T = [\hat{\sigma}^{T}_{1:k} \quad \alpha \hat{\sigma}^{T}_{1:k} H_{12} + (1-\alpha)v_2^T + \alpha \hat{\sigma}_{k + 1} w_2^T]$ 

In [ ]:
import numpy as np
from copy import deepcopy

In [ ]:
def get_H_11_and_H_12(H):
    k = -1
    for i in range(len(H)):
        if not H[i, :].any():
            k = i
            break

    if k == -1:
        return "Error"
    
    H_11 = H[0:k, 0:k]

    H_12 = H[:len(H)-k+1, k:]
    
    return H_11, H_12

In [ ]:
def pagerank(H, v, w, a, conv = 0.01):
    H_11, H_12 = get_H_11_and_H_12(H)
    k = len(H_11)
    
    w_1_T = w[:k].transpose()
    w_2_T = w[k:].transpose()

    v_1_T = v[:k].transpose()
    v_2_T = v[k:].transpose()
    
    sigma = np.random.rand(k+1)
    sigma = sigma / sum(sigma)
    sigma_T = sigma.transpose()
    
    e = np.ones(k)
    
    err = float("inf")
    while err > conv:
        sigma_T_ = deepcopy(sigma_T)
        sigma_T_[:k] = a * sigma_T[:k] @ H_11 + (1 - a) * v_1_T + a * sigma_T[k] * w_1_T
        sigma_T_[k] = 1 - sigma_T_[:k] @ e
        err = sum(abs(sigma_T_ - sigma_T))
        sigma_T = deepcopy(sigma_T_)

    return np.concatenate((sigma_T_[:k], a * sigma_T_[:k] @ H_12 + (1 - a) * v_2_T + a * sigma_T[k] * w_2_T), axis=None)

In [ ]:
H = np.array([
    [0, 0, 1/2, 1/2, 0],
    [0, 0, 1/2, 1/2, 0],
    [1/3, 1/3, 0, 1/3, 0],
    [0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0]
])


w = np.array([
    [0],
    [0],
    [0],
    [1],
    [1],
])

w = w / sum(w)


v = np.array([
    [3],
    [2],
    [2],
    [1],
    [1],
])

v = v / sum(v)


pi = pagerank(H, v, w, 0.5)

pi

array([0.19926792, 0.14371236, 0.19727823, 0.2891157 , 0.17049093])

# Literatura


PageRank Computation, with Special Attention to Dangling Nodes, Ilse C. F. Ipsen, Teresa Selee